### This notebook also contains fine tuning
Advanced version of fine tuning of cnn_5b beginning from feature extraction

In [0]:
  # Observation For fit of feature extraction:
  # Running on TPU with dropout: lets see if this is fast or not by comparing with cnn_5b
  # Result = Too much time 650s vs 30s
  # Now checking GPU with dropout: 20s each with 2100s in the first. This is far better than TPU

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Listing 5.16 - Instantiating the VGG16 convolutional base
import tensorflow
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights = 'imagenet', include_top=False, input_shape=(150,150,3))

# weights: specifies the weight checkpoint from which to initialize the model
# time taken = 5s

58892288/58889256 [==============================] - 1s 0us/step


In [2]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [3]:
# Listing 5.20 Adding a densely connected classifier on top of the convolutional base

from tensorflow.keras import layers, models, optimizers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))   
model.add(layers.Dropout(0.5))              # added by myself
model.add(layers.Dense(1, activation = 'sigmoid'))  

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [4]:
conv_base.trainable

True

In [5]:
# Note: I don't have any other choice than setting conv_base trainable to False
# conv_base.trainable = False, and  No. of weight tensors
print('This is the number of trainable weights '
'before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = False                                  #
print('This is the number of trainable weights '
'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 30
This is the number of trainable weights after freezing the conv base: 4


In [0]:
# Copied from one of the previous cells:
import os
base_dir = 'drive/My Drive/Colab Notebooks/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [7]:
# Listing 5.21 - Training the model end to end with a frozen convolutional base
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,     # changes after trying dropout
    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,    # changes after trying dropout
    class_mode='binary')

model.compile(loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=2e-5),
    metrics=['acc'])

history = model.fit(train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
100/100 [==============================] - 2173s 22s/step - loss: 0.6735 - acc: 0.5920 - val_loss: 0.5088 - val_acc: 0.8020
Epoch 2/30
100/100 [==============================] - 20s 203ms/step - loss: 0.5759 - acc: 0.6990 - val_loss: 0.4249 - val_acc: 0.8480
Epoch 3/30
100/100 [==============================] - 20s 202ms/step - loss: 0.5037 - acc: 0.7610 - val_loss: 0.3781 - val_acc: 0.8600
Epoch 4/30
100/100 [==============================] - 20s 203ms/step - loss: 0.4680 - acc: 0.7770 - val_loss: 0.3535 - val_acc: 0.8650
Epoch 5/30
100/100 [==============================] - 20s 202ms/step - loss: 0.4573 - acc: 0.7870 - val_loss: 0.3364 - val_acc: 0.8690
Epoch 6/30
100/100 [==============================] - 20s 201ms/step - loss: 0.4226 - acc: 0.8030 - val_loss: 0.3168 - val_acc: 0.8720
Epoch 7/30
100/100 [==============================] - 20s 202ms/step - loss: 0.4171 - acc: 0.8120 - val_lo

## Fine-Tuning:

In [8]:
conv_base.trainable

False

In [9]:
# Listing 5.22 - Freezing all layers up to a specific one by block
# set_trainable is just a variable, nothing else

conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

len(model.trainable_weights)    # It must be = 4 + (3*2) = 10

10

In [10]:
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-5),
metrics=['acc'])

history = model.fit(
train_generator,
steps_per_epoch=100,
epochs=100,
validation_data=validation_generator,
validation_steps=50)

Epoch 1/100
100/100 [==============================] - 21s 214ms/step - loss: 0.3182 - acc: 0.8675 - val_loss: 0.2483 - val_acc: 0.8940
Epoch 2/100
100/100 [==============================] - 22s 216ms/step - loss: 0.2717 - acc: 0.8850 - val_loss: 0.2130 - val_acc: 0.9170
Epoch 3/100
100/100 [==============================] - 21s 211ms/step - loss: 0.2621 - acc: 0.8905 - val_loss: 0.1946 - val_acc: 0.9190
Epoch 4/100
100/100 [==============================] - 21s 211ms/step - loss: 0.2312 - acc: 0.9065 - val_loss: 0.2956 - val_acc: 0.8900
Epoch 5/100
100/100 [==============================] - 21s 209ms/step - loss: 0.2302 - acc: 0.9070 - val_loss: 0.1914 - val_acc: 0.9240
Epoch 6/100
100/100 [==============================] - 21s 207ms/step - loss: 0.2125 - acc: 0.9150 - val_loss: 0.2107 - val_acc: 0.9090
Epoch 7/100
100/100 [==============================] - 21s 207ms/step - loss: 0.1940 - acc: 0.9180 - val_loss: 0.2056 - val_acc: 0.9220
Epoch 8/100
100/100 [===========================

In [1]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

NameError: ignored

In [0]:
# graph not showing due to a huge gap of time in running its cell

Validation accuracy is around 92-94%

In [0]:
# Valiation acc not increasing after 36th epoch

In [0]:
# not smoothing the curve now.